In [1]:
# Step 1, download the dataset and save to disk

import requests
import os

dataset_url = "https://raw.githubusercontent.com/sixhobbits/sklearn-intro/master/clickbait.txt"
dataset_filename = "dataset.tsv"

response = requests.get(dataset_url)

with open(dataset_filename, "w") as f:
    f.write(response.text)
    
print("File {} is {} MBs".format(dataset_filename, round(os.path.getsize(dataset_filename)/1024/1024, 2)))

File dataset.tsv is 0.54 MBs


We used the 'requests' library to download the file. It is a 'tab separated' file, so each line contains first the data (the newsheadline) and then a tab character ('/t') and then the label: '1' if the headline should be considered as clickbait and '0' if it is a normal headline. At the end we use the unix 'ls' command to check the file size - it's only 500 KB so we can load the entire file into memory (RAM) at once.

In [2]:
# Step 2, read the TSV back from disk and break our file into data and labels

import csv

data = []
labels = []

with open(dataset_filename) as f:
    reader = csv.reader(f, delimiter="\t")
    for line in reader:
        try:
            data.append(line[0])
            labels.append(line[1])
        except Exception as e:
            print(e)
        

print(data[:3])
print(labels[:3])

["Egypt's top envoy in Iraq confirmed killed", 'Carter: Race relations in Palestine are worse than apartheid', 'After Years Of Dutiful Service, The Shiba Who Ran A Tobacco Shop Retires']
['0', '0', '1']


We break each line into its parts of headline and label, and create two lists. The first item in 'data' list matches with the first label in the 'labels' array, and so on. We print out the first three headlines as a sanity check: the first two are 'normal' headlines (they have '0's in the labels array) and the third one is clickbait (a '1' in the labels array).

In [3]:
%%time

# Step 3, create a vectorizer and transform our documents into TF-IDF vectors
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(data)
print("The dimensions of our vectors:")
print(vectors.shape)
print("- - -")


The dimensions of our vectors:
(10000, 13169)
- - -
CPU times: user 925 ms, sys: 228 ms, total: 1.15 s
Wall time: 1.24 s


We translated our texts into vectors. The final line prints out the dimensions of our vectorized data. The 10000 shows that we have 10000 rows, one for each headline. The 13169 is our vocabulary size: remember in the simple example we looked at previously, we stored some '0's for 'dog' and 'fish' to indicate which words *did not* appear in a specific document. This means that there are 13169 unique words in our dataset of 10000 headlines, and we now need to store an array of size 13167 for each headline. That's not very efficient and nearly all of the items are '0's, as each headline only contains around 6-10 words of the possible 13169.

Luckily sklearn uses an efficient data strucutre automatically: a compressed sparse row matrix or `csr_matrix`.

In [4]:
# Step 4, look at some more statistics and compare how big our data would be in a dense matrix
print("The data type of our vectors")
print(type(vectors))
print("- - -")
print("The size of our vectors (MB):")
print(vectors.data.nbytes/1024/1024)
print("- - -")
print("The size of our vectors in dense format (MB):")
print(vectors.todense().nbytes/1024/1024)
print("- - - ")
print("Number of non zero elements in our vectors")
print(vectors.nnz)
print("- - -")

The data type of our vectors
<class 'scipy.sparse.csr.csr_matrix'>
- - -
The size of our vectors (MB):
0.6759414672851562
- - -
The size of our vectors in dense format (MB):
1004.7149658203125
- - - 
Number of non zero elements in our vectors
88597
- - -


Our matrix stored in sparse format is just over half a megabyte. If we convert this to a dense matrix, in which each zero is explicitly stored, it is over one gigabyte in size! That's around a 1500x increase in size to store exactly the same data.

In [5]:
# Step 5, split our data into train and test sections for training and evaluation 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(vectors, labels, test_size=0.2)

print(X_train.shape)
print(X_test.shape)

(8000, 13169)
(2000, 13169)


In order to check how 'good' our model is, we split our data into two parts: 80% for training our model and 20% for testing its accuracy. Note that we are 'cheating' a little bit as the vectorizer had access to all of the data, training and testing. In a real-world setting, the 'test' data might have some unknown words that were not seen during the training phase. In reality for our project, this makes very little difference.

In [6]:
%%time

# Step 6, train an SVM classifier on the train set

from sklearn.svm import LinearSVC

svm_classifier = LinearSVC()
svm_classifier.fit(X_train, y_train)

predictions = svm_classifier.predict(X_test)

CPU times: user 44.2 ms, sys: 15.2 ms, total: 59.3 ms
Wall time: 91.3 ms


Here we have created a basic SVM classifier and given it the training set and training labels so that it can create a mapping between the two. We then ask it to 'guess' or 'predict' the labels on the test set -- labels it has not seen during training phase. To evaluate it, we have to compare the predictions it made to the 'y_test' labels, which we know are correct. We can do this most simply by looking at 'accuracy', but other important metrics are 'precision' and 'recall', which describe not only how *often* the algorithm made a mistake, but also what *kinds* of mistakes it tends to make: either classifying something as clickbait when it is actually normal, or vice versa. The model only took 44 ms to train on 8000 examples and make 2000 predictions!

In [7]:
# Step 7, evaluate the model

from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))


0.961
              precision    recall  f1-score   support

           0       0.95      0.98      0.96      1014
           1       0.97      0.95      0.96       986

    accuracy                           0.96      2000
   macro avg       0.96      0.96      0.96      2000
weighted avg       0.96      0.96      0.96      2000



We achieved 96% accuracy, which is surprisingly high considering that the problem is not that easy and we only used the default configurations of our tools. We can see from the longer report that the algorithm is fairly good at all metrics -- because all of the numbers are the same, the model doesn't have specific weaknesses.

In [8]:
%%time

# Step 8, compare our results to those from a Neural Network model
from sklearn.neural_network import MLPClassifier
mlp_classifier = MLPClassifier(max_iter=20)
mlp_classifier.fit(X_train, y_train)
predictions = mlp_classifier.predict(X_test)
print(accuracy_score(y_test, predictions))

0.966
CPU times: user 56.2 s, sys: 9.47 s, total: 1min 5s
Wall time: 34.3 s


/usr/local/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


We can see that the neural network achieved very similar results to the SVM, but it took a lot longer to train (nearly a minute instead of a few milliseconds, in spite of the fact that we limited the number of iterations to 20 instead of the default of 200). There are a lot of so-called 'hyperparameters' in neural networks to tweak to try and get a good balance of performance and required resources. We briefly mentioned batch_size, max_iterations, and learning_rate previously: feel free to try adjusting these based on the examples in https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html to see how they affect the training time, ram use, and accuracy.

In [9]:
# Step 9, persisiting our models to disk and comparing file sizes
import pickle

with open("vectorizer.pickle", "wb") as f:
    f.write(pickle.dumps(vectorizer))
    print("File {} is {} MB".format("vectorizer", round(os.path.getsize("vectorizer.pickle")/1024/1024, 2)))
    
with open("svm_classifier.pickle", "wb") as f:
    f.write(pickle.dumps(svm_classifier))
    print("File {} is {} MB".format("svm classifier", round(os.path.getsize("svm_classifier.pickle")/1024/1024, 2)))

with open("mlp_classifier.pickle", "wb") as f:
    f.write(pickle.dumps(mlp_classifier))
    print("File {} is {} MB".format("mlp classifier", round(os.path.getsize("mlp_classifier.pickle")/1024/1024, 2)))


File vectorizer is 0.8 MB
File svm classifier is 0.1 MB
File mlp classifier is 30.15 MB


We can use Python's `pickle` library to save our models and vectorizer to disk. We would need both to analyse any more headlines outside of the dataset we already used -- first we would have to vectorize the new headlines with the same vectorizer we used during training (so that the word mapping still mean the same things), and then we would have to load our classifier back into memory and get it to make predictions on the new data. 

We can see that the svm classifier is not only significantly faster to train, but it is also much smaller -- only 0.1 MB. The MLP classifier by contrast is over 30 MB. 

In [23]:
%%time
import csv

tweets = []

with open("tweets.csv") as f:
    reader = csv.reader(f)
    next(reader)
    
    for line in reader:
        tweet = line[0].lower()
        tweets.append(tweet)

CPU times: user 5.83 s, sys: 280 ms, total: 6.11 s
Wall time: 6.16 s


In [26]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer

char_vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectors = char_vectorizer.fit_transform(tweets)

CPU times: user 1min 28s, sys: 4.91 s, total: 1min 32s
Wall time: 1min 37s


In [39]:
%%timeit
import h5py

with h5py.File('vectors.hdf5', 'w') as f:
    dset = f.create_dataset("default", data=vectors.data)

797 ms ± 168 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%%timeit
with open("vectors.pickle", "wb") as f:
    f.write(pickle.dumps(vectors.data))
    # print("File {} is {} MB".format("vectorizer", round(os.path.getsize("vectors.pickle")/1024/1024, 2)))

1.07 s ± 169 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
